# InvokeAI in Google Colab

###Introduction

This is a tool to use Google Colab to run the AI image generation tool: Invokeai (https://invoke-ai.github.io/InvokeAI/). <br />
This can save the generated images in Google Drive or work purely in the runtime. <br />
You can optionally save your models in Google Drive too, This takes up far more space but makes resuming generation very simple. <br />

A GPU should be enabled by default, but if not the setting can be found in the menu under: Edit > Notebook Settings > Hardware accelerator > T4 GPU

To start, Click "Runtime" > "Run All". Alternatively you can click the "play" button on each step below sequentially; there's no need to wait for a step to complete before starting the next, as they will be added to an execution queue.

This uses the "Low VRAM mode" included with InvokeAI 5.5.0+

In [ ]:
#@title 1. Configuration { display-mode: "form" }
#@markdown #Instance Type
#@markdown **Google_Drive** = Stores Models, Images and the database in your Google drive, the space it takes up is heavily based on the type and number of models that you are using. <br>
#@markdown **Temporary** = Everything is stored in the runtime and is removed when the runtime ends or crashes, make sure to download your images! <br>
Type = "Google_Drive" #@param ['Google_Drive','Temporary'] {type:"string"}
#@markdown If using "Google Drive" mode, where do you want the models saved to? <br>
#@markdown I recommend also using Google_Drive, however if you have limited space and want to only save images to Drive, you can change that here!
Models = "Google_Drive" #@param ['Google_Drive','Temporary'] {type:"string"}
#@markdown Note: if you don't save models to drive and you re-load the same instance the models will think they are still there, you have to remove + re-download them.

#@markdown ---

#@markdown #Connection Type.
#@markdown **NGROK**: (Recommended) Highly stable but needs a little setting up, An NGROK token is required, sign up for free and get one here: https://dashboard.ngrok.com/get-started/your-authtoken - Once you have the token, please put it in below.<br>
#@markdown **NGROK_APT**: An alternate version of NGROK that runs as a Linux service rather than a python service.<br>
#@markdown **Localtunnel**: Slower than NGROK and more often has issues, but will just work, no token or configuration needed.<br>
connection_type = "Localtunnel"  #@param ["Localtunnel","NGROK","NGROK_APT"]
ngrok_token = "None" #@param ['None'] {allow-input: true}

#@markdown ---
#@markdown #Instance Selector.
#@markdown If you would like to have more than one instance of InvokeAI so you can keep things separate you can do this here: <br>
#@markdown "Default" Keeps the files either in /content/invokeai or /content/drive/MyDrive/InvokeAI <br>
#@markdown Changing this to anything else puts the files in a sub-folder of that location called "CustomInstances" and then the Instance name.
Instance = "Default"  #@param ["Default","Anime","Photorealism"] {allow-input: true}


#@markdown ---
#@markdown #InvokeAi Version.
#@markdown The default version is "the latest stable release of InvokeAI" If you want to specify a specific version, please input that here:. <br>
#@markdown Any version 5.7.2+ should work with all features, anything before 5.0.0 will probably not work at all.
version = "Default"  #@param ["Default","5.7.2","5.6.1"] {allow-input: true}

#@markdown ---
#@markdown #Model Management.
#@markdown All model management is all done in-app, the "Model Manager" is found on the left hand side once you are in the app. <br />
#@markdown If you are using Temporary Mode, and want to attach Google Drive purely to import models from, tick this box.
GDrive_Import = False #@param {type: "boolean"}
#@markdown The path to the root of your Google drive will be added as: /content/drive/MyDrive/ You will use that in the "Model manager" to import models.


#@markdown ---
#@markdown #Memory Management.
#@markdown InvokeAI offers two memory allocator methods to load and manage models in VRAM. Typically, the CUDA allocator outperforms the pytorch allocator, reducing peak VRAM usage. In most cases, CUDA should improve generation speeds. <br />
#@markdown Note: InvokeAI Version 5.7.2+ only (Please use 'pytorch' for older versions) <br />
#@markdown Note2: If you for whatever reason want to use CPU mode... Please choose "pytorch" as CUDA will not work.
MemoryManagementMethod = "CUDA"  #@param ["CUDA","pytorch"]






In [ ]:
#@title 2. Installation. { display-mode: "form" }
#@markdown Install InvokeAI and configure it as requested, This takes around 4-5 mins.

from IPython.display import clear_output

#Setting File Path
file_path = '/content/invokeai'

#Set up temporary storage if running in "Temporary" mode.
if Type == "Temporary":
    import os
    if not os.path.exists(file_path):
      os.makedirs(file_path)

    #Mount google drive for model imports if requested.
    if GDrive_Import == True:
      import os
      from google.colab import drive
      if not os.path.exists('/content/drive/'):
        print("Connecting to Google Drive, please log in using the pop-up window.")
        drive.mount('/content/drive')
      print("Connected!")

# Mount and set up Google drive if running in "Google_Drive" mode.
if Type == "Google_Drive":
  import os
  from google.colab import drive
  if not os.path.exists(file_path):
    print("Connecting to Google Drive, please log in using the pop-up window.")
    drive.mount('/content/drive')
    if not os.path.exists(file_path):
      os.makedirs(file_path)
  print("Connected!")

#set working DIR
%cd {file_path}

#Clear Output
clear_output()

print("=======================")
print("|Install Dependencies.|")
print("=======================")

#Update pip
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python -m pip install --upgrade pip

#Create InvokeAI root
import os
os.environ['INVOKEAI_ROOT'] = file_path
if not os.path.exists(file_path):
  os.makedirs(file_path)

#Confirm that This is working in the correct place.
%cd {file_path}

#Clear Output
clear_output()


print(" ")
print("======================")
print("|1. Install InvokeAI.|")
print("======================")

#Version selector
if version == "Default":
  print(" ")
  print("Installing the Latest stable release of InvokeAI. This may take 4-5 minutes. Please be patient...")
  !pip install invokeai[xformers] &> /dev/null
  print("Installation command issued.")
if version != "Default":
  print(" ")
  print(f"Installing InvokeAI version: {version}. This may take 4-5 minutes. Please be patient...")
  !pip install invokeai[xformers]=={version} &> /dev/null
  print("Installation command issued.")

print(" ")
print("App Installed.")

#This 'Deactivate' is needed in case you run this twice in one session where an update is released between runs, it will error 99% of the time.
if os.path.exists("/content/RunSuccess"):
  print(" ")
  print("This Deactivate only is needed on subsequent runs. Ignore that it has errored. It will most of the time, and is a cleanup.")
  !deactivate
  print("Deactivate command has been executed.")
if not os.path.exists("/content/RunSuccess"):
  !touch "/content/RunSuccess"

print(" ")
print("Expected Error saying path does not exist:") # Removed extra space
!source .venv/bin/activate


print(" ")
print("========================")
print("|2. Configure InvokeAI.|")
print("========================")

#Settings for ALL instances.
!echo "# Internal metadata - do not edit:" > /content/invokeai/invokeai.yaml
!echo "schema_version: 4.0.2" >> /content/invokeai/invokeai.yaml
!echo " " >> /content/invokeai/invokeai.yaml
!echo "# Put user settings here - see https://invoke-ai.github.io/InvokeAI/configuration/" >> /content/invokeai/invokeai.yaml

#Default Instance location.
if Instance == "Default":
  if Type == "Google_Drive":
    print(" ")
    print("Configuring the runtime for 'Google Drive' storage.")

    if Models == "Google_Drive":
      !echo "models_dir: /content/drive/MyDrive/InvokeAI/Models" >> /content/invokeai/invokeai.yaml
    !echo "outputs_dir: /content/drive/MyDrive/InvokeAI" >> /content/invokeai/invokeai.yaml
    !echo "db_dir: /content/drive/MyDrive/InvokeAI/DB" >> /content/invokeai/invokeai.yaml
    !echo "enable_partial_loading: true" >> /content/invokeai/invokeai.yaml
    if MemoryManagementMethod == "CUDA":
      !echo "pytorch_cuda_alloc_conf: \"backend:cudaMallocAsync\"" >> /content/invokeai/invokeai.yaml
    print(" ")
    print("Locations set for Google Drive!")

  if Type == "Temporary":
    print(" ")
    print("Configuring the runtime for 'Temporary' storage.")

    !echo "enable_partial_loading: true" >> /content/invokeai/invokeai.yaml
    if MemoryManagementMethod == "CUDA":
      !echo "pytorch_cuda_alloc_conf: \"backend:cudaMallocAsync\"" >> /content/invokeai/invokeai.yaml
    print(" ")
    print("Locations set for Temporary Storage!")

#Non-Default Instance location.
if Instance != "Default":
  if Type == "Google_Drive":
    print(" ")
    print("Configuring the runtime for 'Google Drive' storage, with custom instance.")
    print(" ")
    print("Instance name:")
    print(Instance)

    if Models == "Google_Drive":
      !echo "models_dir: /content/drive/MyDrive/InvokeAI/CustomInstances/{Instance}/Models" >> /content/invokeai/invokeai.yaml
    !echo "outputs_dir: /content/drive/MyDrive/InvokeAI/CustomInstances/{Instance}" >> /content/invokeai/invokeai.yaml
    !echo "db_dir: /content/drive/MyDrive/InvokeAI/CustomInstances/{Instance}/DB" >> /content/invokeai/invokeai.yaml
    !echo "enable_partial_loading: true" >> /content/invokeai/invokeai.yaml
    if MemoryManagementMethod == "CUDA":
      !echo "pytorch_cuda_alloc_conf: \"backend:cudaMallocAsync\"" >> /content/invokeai/invokeai.yaml
    print(" ")
    print("Locations set for Google Drive!")


  if Type == "Temporary":
    print(" ")
    print("Configuring the runtime for 'Temporary' storage, with custom instance.")
    print(" ")
    print("Instance name:")
    print(Instance)

    !echo "models_dir: /content/invokeai/CustomInstances/{Instance}/Models" >> /content/invokeai/invokeai.yaml
    !echo "outputs_dir: /content/invokeai/CustomInstances/{Instance}" >> /content/invokeai/invokeai.yaml
    !echo "db_dir: /content/invokeai/CustomInstances/{Instance}/DB" >> /content/invokeai/invokeai.yaml
    !echo "enable_partial_loading: true" >> /content/invokeai/invokeai.yaml
    if MemoryManagementMethod == "CUDA":
      !echo "pytorch_cuda_alloc_conf: \"backend:cudaMallocAsync\"" >> /content/invokeai/invokeai.yaml
    print(" ")
    print("Locations set for Temporary Storage!")

#Misc Fixes
print(" ")
print("================")
print("|3. Misc Fixes.|")
print("================")

#Reinstall some features.
print(" ")
print("Reinstalling some pip modules to make sure they function correctly.")

#These ones were previously needed but break things now, leaving as comment for now.
#!pip uninstall -y opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp pydantic &> /dev/null
#!pip install opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp pydantic &> /dev/null

!pip uninstall -y fastapi-events fastapi &> /dev/null
!pip install fastapi-events fastapi &> /dev/null
!pip install --upgrade jax_cuda12_plugin &> /dev/null


#Success message
print(" ")
print("================================")
print("|5. Done! Launch the app below!|")
print("================================")
print(" ")
print("Note: When you start the app, it may throw the error shown below. This is often benign, and the application should still function correctly. Please ignore this specific error if the app works.")
print("/bin/bash: line 1: /content/invokeai/.venv/bin/activate: No such file or directory")



In [ ]:
#@title 3. Start InvokeAI. { display-mode: "form" }
#@markdown ## Starting the App
#@markdown This step takes about 15 seconds to generate your URL but 30 seconds after it is launched before it will work fully! <br>

#@markdown ## Notes about connection types.
#@markdown **NGROK** (Recommended) = Very stable but requires a token, see the "configuration" step for more details.<br>
#@markdown **NGROK_APT** = An alternate version of NGROK that runs using the system packages rather than Python. <br>
#@markdown **Localtunnel** = Once it gets going it is quite stable but often has "502" Errors, You must wait for the Localtunnel service to resolve the issue, or try another connection type.<br>

#@markdown Note: If you are using "Google_Drive" It is STRONGLY recommended to stop this step from running when you have finished, so it gracefully shuts down.<br>
#@markdown If you just "Disconnect and Delete this runtime" it may cause some corruption if all information has not finished being uploaded to your drive yet.

!source .venv/bin/activate

%cd {file_path}
import os

#Clear Output
clear_output()

if connection_type == "Localtunnel":
  print("How to connect to localtunnel:")
  print("A localtunnel Interface connection is generated here, To use this, please do the following:")
  print("1. Copy this IP address")
  !curl ipv4.icanhazip.com
  print("2. Click the random 'https://XXX-YYY-ZZZ.loca.lt' link that is generated below.")
  print("3. Paste the IP into the provided box and submit. ")
  print(" ")
  print("Note: An error of '502 Bad Gateway' typically is an error at Localtunnel's end. A '504 Gateway Time-out' Error means invokeai has not started yet.")
  print(" ")
  !npm install -g localtunnel
  !lt --port 9090 & ! . {file_path}/.venv/bin/activate; invokeai-web


#NGROK_Python
if connection_type == "NGROK":
  if ngrok_token == "None":
    print("You have selected NGROK but did not supply an NGROK token.")
    print("Falling back to a 'Localtunnel' connection type.")
    print("Please either add an NGROK token to step 1, re-run step 1, then re-run this step, or just re-run this step to use 'Localtunnel'.")
    connection_type = "Localtunnel"
  if ngrok_token != "None":
    !pip install pyngrok --quiet
    from pyngrok import ngrok
    ngrok.kill()
    ngrok.set_auth_token(ngrok_token)
    public_url = ngrok.connect(9090).public_url
    print(f'InvokeAI Public URL: {public_url}')
    ! . {file_path}/.venv/bin/activate; invokeai-web

#NGROK_APT
if connection_type == "NGROK_APT":
  if ngrok_token == "None":
    print("You have selected NGROK but did not supply an NGROK token.")
    print("Falling back to a 'Localtunnel' connection type.")
    print("Please either add an NGROK token to step 1, re-run step 1, then re-run this step, or just re-run this step to use 'Localtunnel'.")
    connection_type = "Localtunnel"
  if ngrok_token != "None":
    !curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null
    !echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list
    !sudo apt update
    !sudo apt install ngrok
    !ngrok config add-authtoken {ngrok_token}
    clear_output()
    !echo "You can find the connection URL here in the NGROK portal:"
    !echo " https://dashboard.ngrok.com/endpoints" # Added space
    !nohup ngrok http http://localhost:9090 &
    ! . {file_path}/.venv/bin/activate; invokeai-web

# Model Training<br />
hollowstrawberry has an amazing Google Colab LoRA maker, it is much better than I could do! It can be found here:<br />
Dataset Maker - https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb <br />
LoRA Trainer - https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb

# FAQ:

## Images are taking a LONG time to generate. (~30 mins for a 512x512 image.)
You may have used up your compute allowance and are running in CPU mode. While this is running, clicking the "RAM / Disk" button in the top right to see if it lists "GPU RAM". If not, you are not connected to a GPU instance.
or maybe, you have the GPU turned off, check "Edit > Notebook settings" to make sure a GPU is selected. ("T4 GPU" is the current default)

## It is taking a while to even start loading.
If you have started a new instance / are changing a model, the model needs to be loaded into Invoke. If you are using "Google Drive" it has to download the model to the runtime, this can take a while especially for XL (5-10 mins) or FLUX models (20+ mins).

##No link is given to me?
You can check the status of the service at one of these two pages. <br>
Localtunnel: https://downforeveryoneorjustme.com/localtunnel.me <br>
NGROK: https://status.ngrok.com


## I get an error "You have recently exceeded an allowance, most recently at (Time)"
This is typically when you have exceeded some Google restriction(s). This is most common when you stop / start instances frequently and often does not affect anything. <br>
If it is blocking things, you typically have to wait 1 hour for soft allowances to reset.

##How much longer can my instance run for today?
If you click the "RAM / Disk" button in the top right it will tell you something like "At your current usage level, this runtime may last up to 1 hour 20 minutes"

## My instance disconnects while I'm using it.
Technically using Web interface to run programs in a way like this project does, is against Google Colab's Terms of Service. If you don't look at this tab for 15+ minutes it may hibernate. Simply flick back to this tab on a semi-regular basis to keep it active!

## Are there any known issues?
If there are, I will typically update the header at the top of this page above the "Introduction".

## I want to make my own changes to this file, and save them.
Feel free to! "File > Save a copy in drive" is the safest way to do this.

## Is this maintained frequently?
I am a solo developer that maintains it the best that I can, I do use this project frequently so hopefully I will notice any major issues. <br>
If you have any issues you can raise them on Github, I will deal with them when I can! <br>
https://github.com/MikeEmmett/InvokeAI/issues

## I am using Google Drive and I want to reset everything.
If you log onto https://drive.google.com/drive/my-drive with your Google account you will find a folder called "InvokeAI" <br>
If you just want to reset it without keeping anything, simply delete the whole folder. <br>
If you want to reset it but keep the models / Images in an archive, rename the folder.

## I don't understand the options above...
Experiment! This will work fully if you just do "Runtime > Run all". It will connect to your Google Drive and spin up a "Localtunnel" connection, and you simply copy the IP address / paste into the link as instructed in the last step! Also feel free to play around, you can't break things permanently!